<a href="https://colab.research.google.com/github/jamesburrell2/NBXplorer/blob/master/ResNet50_Classifier_Project_(IBM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building a classifier with ResNet50

In [2]:
!pip install skillsnetwork
import skillsnetwork

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.2 MB/s eta 0:00:00


In [3]:
import keras #import Keras library
from keras.preprocessing.image import ImageDataGenerator #Image Datagenerator

In [4]:
from keras.models import Sequential #More of the Kera library
from keras.layers import Dense

In [5]:
from keras.applications import ResNet50 #ResNet50 is a model to build classifier
from keras.applications.resnet50 import preprocess_input

ModuleNotFoundError: ignored

In [9]:
from keras.applications import ResNet50 #ResNet50 Model

In [13]:
from keras.applications.resnet import ResNet50 #Workaround to import all of ResNet

In [14]:
#Get the data for the project
await skillsnetwork.prepare("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip", overwrite=True)

  0%|          | 0/80037 [00:00<?, ?it/s]

Saved to '.'


Define Global Constants after uploading data
Only 2 classes of data so num_classes =2
ResNet50 model use 224x224 image size so resize to model's parameter
Using 100 image batches

In [15]:
#Number of classes is two
num_classes = 2

image_resize =224

batch_size_training = 100
batch_size_validation = 100

In [18]:
from keras.applications.resnet import preprocess_input

In [19]:
#Construct ImageDataGenerator Instances

data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [20]:
#Use flow_from_directory to get training images
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical'
)

Found 30001 images belonging to 2 classes.


In [21]:
#Use flow_from_directory to get validation images
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical'
)

Found 10001 images belonging to 2 classes.


In [22]:
#Build, Compile and Fit Model

model = Sequential()

In [23]:
# Adding ResNet50 pre-trained model to our model. But we don't want to include the pre-trained model's output layer;
# We'd rather define our own output layer (top layer)
# To remove pre-trained model's output layer, set "include_top" to False

model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
))

94765736/94765736 [==============================] - 0s 0us/step


In [24]:
model.add(Dense(num_classes, activation='softmax'))

In [25]:
#To access model's layers, use layers attribute of our model object
model.layers

In [26]:
#To see ResNet50 layers run
model.layers[0].layers

In [27]:
#We now only need to train our Dense layer since model is pretrained on other layers
model.layers[0].trainable=False

#Summary attribute is called by the following
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [28]:
# Compile model using adam optimizer

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
# Define how many steps compose an epoch
# Typically No of Images/batch_size

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [30]:
#YAY!! Let's train our model
# We're using fit_generator method

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

<ipython-input-30-1c015ae4f7b7>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(


Epoch 1/2
301/301 [==============================] - 5330s 18s/step - loss: 0.0239 - accuracy: 0.9920 - val_loss: 0.0056 - val_accuracy: 0.9989
Epoch 2/2
301/301 [==============================] - 5233s 17s/step - loss: 0.0049 - accuracy: 0.9991 - val_loss: 0.0039 - val_accuracy: 0.9994


In [31]:
# Save models that you may use again after training

model.save('Classifier_resnet_model.h5')